In [2]:
import pandas as pd

In [13]:
data = pd.read_csv("data/notes.tsv", sep="\t")
print(data.shape)

C:\Users\cdemm\AppData\Local\Temp\ipykernel_4196\668006533.py:1: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/notes-00000.tsv", sep="\t")


(1437535, 23)


## Creating data subset for annotation